## Title:

### Introduction:

Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal.

Clearly state the question you will try to answer with your project

Identify and describe the dataset that will be used to answer the question

### Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 

Clean and wrangle your data into a tidy format

Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 

Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

### Methods:
Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?

Describe at least one way that you will visualize the results
Expected outcomes and significance:

What do you expect to find?

What impact could such findings have?

What future questions could this lead to?

### Expected outcomes and significance:
What do you expect to find?

What impact could such findings have?

What future questions could this lead to?

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [59]:
# Rana attempting to read data take 1

weather_data_raw <- read_csv("data/daily_weather_2020.csv") %>%
    select(-X1) %>%
     rename(Country = "Country/Region") # remove first column, it only contains the number of rows, will prob need to remove many more cols 
    #filter(Country/Region == "??")

names <- colnames(weather_data_raw) # So that we can see all the column names in the dataframe

count <- weather_data_raw %>%
            group_by(Country) %>%
            summarize(number_of_rows = n()) %>%
            arrange(desc(number_of_rows)) # So that we can see the rows of data we have to work with in each region         

weather_data <- weather_data_raw %>%
                    select(Country, precipProbability, temperatureHigh, temperatureLow, dewPoint, humidity, pressure) %>%
                    filter(Country == "US") 
weather_data

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_double(),
  `Country/Region` = col_character(),
  `Province/State` = col_character(),
  time = col_date(format = ""),
  summary = col_character(),
  icon = col_character(),
  precipType = col_character()
)

See spec(...) for full column specifications.

`summarise()` ungrouping output (override with `.groups` argument)



Country,precipProbability,temperatureHigh,temperatureLow,dewPoint,humidity,pressure
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
US,0.04,60.37,32.53,32.58,0.63,1019.4
US,0.07,61.00,47.42,30.54,0.61,1018.4
US,0.96,68.09,61.20,54.40,0.88,1011.2
US,0.98,63.40,54.90,61.56,0.95,1011.4
US,0.37,59.38,30.98,43.43,0.71,1019.7
US,0.06,60.45,35.99,30.59,0.64,1027.8
US,0.06,67.34,50.55,42.38,0.76,1022.4
US,0.12,62.47,31.89,37.19,0.57,1023.2
US,0.03,63.61,34.41,29.19,0.57,1030.1


In [27]:
# For self reference... na.omit(df) can be used to remove rows with NA values